In [ ]:
!pip install sre-yield pythainlp

     |████████████████████████████████| 11.1MB 8.1MB/s 
     |████████████████████████████████| 1.5MB 36.0MB/s 
     |████████████████████████████████| 61kB 7.9MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449905 sha256=1da972553c17424333c73ffd46f9856adc1fe3248ad654f6a9d1a88d2ae07ed4
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk
ERROR: google-colab 1.0.0 has requirement requests~=2.21.0, but you'll have requests 2.22.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
  Found existing installation: requests 2.21.0
    Uninstalling requests-2.21.0:
      Successfully uninstalled requests-2.21.0


In [ ]:
RE_TCC = (
    """\
เc็c
เcctาะ
เccีtยะ
เccีtย(?=[เ-ไก-ฮ]|$)
เcc็c
เcิc์c
เcิtc
เcีtยะ?
เcืtอะ?
เc[ิีุู]tย(?=[เ-ไก-ฮ]|$)
เctา?ะ?
cัtวะ
c[ัื]tc[ุิะ]?
c[ิุู]์
c[ะ-ู]t
c็
ct[ะาำ]?
แc็c
แcc์
แctะ
แcc็c
แccc์
โctะ
[เ-ไ]ct
""".replace(
        "c", "[ก-ฮ]"
    )
    .replace("t", "[่-๋]?")
    .split()
)

In [ ]:
PAT_TCC = "|".join(RE_TCC)

In [ ]:
import sre_yield

In [ ]:
#th=list(sre_yield.AllStrings(PAT_TCC))

In [ ]:
#len(th)

In [ ]:
from pythainlp import thai_characters,thai_letters

In [ ]:
a="[ก-ฮ]"#"[ขคฆดจฉชซฌฎฏฑฒตทธศษสปพฟภญณรลฬ]"

In [ ]:
RE_ETCC = (
    """\
FV+C+UV1+([UV2]?)+RC
FV+C+([C]?)+RV1+RV2
C+UV1+([UV2]?)+RC
FV+C+UV+RC
C+((C+((UV1)?)+((LV)?)+UV2)?)
""".replace("RC",a)
    .replace("C","[ก-ฮ]")
    .replace("RV1","า")
    .replace("RV2","ะ")
    .replace("UV1","ิ")
    .replace("UV2","์")
    .replace("UV","[ิีื๊็์่้๊๋]")
    .replace("FV","[เแโไใ]")
    .replace("LV","[ุู]")
    .replace("RV","[ะาๆฯๅำ]")
    .replace("+","")
    .split()
)

In [ ]:
PAT_ETCC = "|".join(RE_ETCC)

In [ ]:
PAT_ETCC

'[เแโไใ][ก-ฮ]ิ([์]?)[ก-ฮ]|[เแโไใ][ก-ฮ]([[ก-ฮ]]?)าะ|[ก-ฮ]ิ([์]?)[ก-ฮ]|[เแโไใ][ก-ฮ][ิีื๊็์่้๊๋][ก-ฮ]|[ก-ฮ](([ก-ฮ]((ิ)?)(([ุู])?)์)?)'

In [ ]:
th2=list(set(list(sre_yield.AllStrings(PAT_ETCC))))

In [ ]:
len(th2)

144394

In [ ]:
with open("etcc.dict","w",encoding="utf-8") as f:
  f.write("\n".join(th2))

In [ ]:
th2[100]

'ไค์ฃ'

In [ ]:
"แร" in th2

False

In [ ]:
from pythainlp.tokenize import Tokenizer,Trie

In [ ]:
trie = Trie(th2)

In [ ]:
cut=Tokenizer(trie, engine='longest')

In [ ]:
cut.word_tokenize("เด็กแข็งแรง")

['เด็ก', 'แข็ง', 'แร', 'ง']

In [ ]:
import re
def cut_subword(listword):
  j = len(listword)
  i = 0
  while True:
    if i == j:
      break
    if re.search("[ะาๆฯๅำ]", listword[i]) and i>0 and len(listword[i])==1:
      listword[i-1]+=listword[i]
      del listword[i]
      j-=1
    i+=1
  return listword

In [ ]:
cut_subword(cut.word_tokenize("หาเงินเพื่อเรียน"))

['หา', 'เงิน', 'เพื่', 'อ', 'เรีย', 'น']

In [ ]:
cut.word_tokenize("หนังสือ")

['ห', 'นัง', 'สือ']

In [ ]:
cut.word_tokenize("เพราะเกาะ")

['เพราะ', 'เกาะ']

In [ ]:
cut.word_tokenize("แพทย์เพาะพันธุ์ยีนส์")

['แพ', 'ท', 'ย์เพาะ', 'พันธุ์', 'ยี', 'นส์']

In [ ]:
cut_subword(cut.word_tokenize("ตากลม"))

['ตา', 'ก', 'ล', 'ม']

## Test

In [ ]:
'/'.join(cut.word_tokenize("หอมรดกไทย")) # ห/อ/ม/ร/ด/ก/ไท/ย

'ห/อ/ม/ร/ด/ก/ไท/ย'

In [ ]:
'/'.join(cut.word_tokenize("เรือน้อยลอยอยู่")) # เรือ/น้/อ/ย/ล/อ/ย/อ/ยู่

'เรือ/น้/อ/ย/ล/อ/ย/อ/ยู่'

In [ ]:
'/'.join(cut_subword(cut.word_tokenize("ประสานงานกับลูกค้า"))) # ป/ระ/สา/น/งา/น/กับ/ลู/ก/ค้า

'ป/ระ/สา/น/งา/น/กับ/ลู/ก/ค้า'

In [ ]:
'/'.join(cut_subword(cut.word_tokenize("ตากลม"))) # ตา/ก/ล/ม

'ตา/ก/ล/ม'

In [ ]:
'/'.join(cut_subword(cut.word_tokenize("หาเงินเพื่อเรียน"))) # หา/เงิน/เพื่/อ/เรีย/น

'หา/เงิน/เพื่/อ/เรีย/น'